### 할로윈 투자전략
1. 11월 첫날의 주식을 매수 -> 6개월간 유지 4월 말일에 판매
2. 11월 첫날의 시가로 구매
3. 4월 말에 종가 판매 
4. 수익율 -> 3번과정의 종가 / 2번 과정의 시가
5. 데이터를 이용하여 수익율 계산

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv('../../csv/AMZN.csv')

In [ ]:
df.head(1)

In [ ]:
# date 컬럼을 시계열 변경 -> Date 컬럼을 인덱스로 변환
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
# 2000년 11월 데이터를 확인
df.loc['2000-11'].head(1)
# df.loc['2000-11'].iloc[0, 0]

In [ ]:
# 2001년 4월 데이터를 확인 
df.loc['2001-04'].tail(1)['Close']
df.loc['2001-04'].iloc[-1]['Close']

In [ ]:
# 2000년부터 2019년까지의 11월 첫날의 데이터와 4월의 마지막날의 데이터만 추출해서 
# 새로운 데이터프레임을 생성

# 비어있는 데이터프레임을 생성 
price_df = pd.DataFrame()

# 반복문을 이용해서 2000년부터 2019년까지 반복
for i in range(2000, 2019):
    # 11월 첫날의 데이터를 추출 
    buy_mon = str(i) + '-11'  ## 2000-11, 2001-11, ...
    buy = df.loc[buy_mon].head(1)
    # 11월 첫날의 데이터를 데이터프레임에 대입
    price_df = pd.concat([price_df, buy] , axis=0)
    sell_mon = str(i+1) + '-04' ## 2001-04, 2002-04, ....
    sell = df.loc[sell_mon].tail(1)
    # 4월 마지막 날의 데이터를 데이터프레임에 대입
    price_df = pd.concat([price_df, sell], axis=0)

price_df

In [ ]:
# Open, Close를 제외한 나머지 컬럼을 모두 제거 
price_df2 = price_df[['Open', 'Close']]

In [ ]:
# 수익율 계산 -> 4월에 계산

# 수익율 컬럼을 생성 
price_df2['rtn'] = 1

In [ ]:
price_df2.iloc[1]['rtn']

In [ ]:
# 위치의 값이 홀수인 경우 수익율을 계산하여 대입 
for i in range(len(price_df2)):
    if i % 2 == 1:
        rtn = price_df2.iloc[i]['Close'] / price_df2.iloc[i-1]['Open']
        price_df2.iloc[i, 2] = rtn


In [ ]:

price_df2.head()

In [ ]:
price_df2['rtn2'] = 1

In [ ]:
for i in range(1, len(price_df2), 2):
    rtn = price_df2.iloc[i]['Close'] / price_df2.iloc[i-1]['Open']
    price_df2.iloc[i, 3] = rtn

In [ ]:
price_df2.head()

In [ ]:
# 인덱스의 값으로 조건식을 구성하여 수익율 계산

price_df2.index.month == 4

In [ ]:
price_df2['rtn3'] = 1

In [ ]:
price_df2.shift(1)

In [ ]:
for i in price_df2.index:
    # i? -> price_df2의 인덱스 값들이 하나씩 대입 -> 시계열데이터
    # print(i)
    if i.month == 4:
        rtn = price_df2.loc[i, 'Close'] / price_df2.shift(1).loc[i, 'Open']
        price_df2.loc[i, 'rtn3'] = rtn

price_df2.loc[price_df2.index.month == 4]

In [ ]:
# 누적 수익율을 계산

acc_rtn = 1

for i in price_df2.index:
    rtn = price_df2.loc[i, 'rtn']
    acc_rtn *= rtn

acc_rtn

In [ ]:
price_df2['rtn'].cumprod()

In [56]:
from dateutil.relativedelta import relativedelta

In [69]:
# 2000년도 11월 1일 시계열 데이터를 생성 
start = datetime(year=2000, month=11, day=1)

In [70]:
# 시작일에서 5개월 뒤 
end = start + relativedelta(months=5)

In [73]:
buy_mon = start.strftime('%Y-%m')

In [74]:
sell_mon = end.strftime('%Y-%m')

In [96]:
# 함수를 생성 
# 매개변수의 개수는? 데이터프레임, 시작년도, 종료년도, 시작월
def six_month(_df, _start, _end, _month):
    # 인덱스가 Date인지 확인하는방법?
    if 'Date' in _df.columns:
        # Date가 컬럼에 존재하는경우 
        # Date 컬럼을 시계열 데이터로 변환
        _df['Date'] = pd.to_datetime(_df['Date'])
        _df.set_index('Date', inplace=True)
    else:
        _df.index = pd.to_datetime(_df.index)

    # 누적수익율 데이터를 생성 
    acc_rtn = 1

    for i in range(_start, _end):
        start = datetime(year = i, month = _month, day = 1)
        end = start + relativedelta(months=5)

        # 구매하는 달 
        buy_mon = start.strftime('%Y-%m')
        # 판매하는 달
        sell_mon = end.strftime('%Y-%m')

        buy = _df.loc[buy_mon].iloc[0]['Open']
        sell = _df.loc[sell_mon].iloc[-1]['Close']

        rtn = sell / buy
        # print(f"매수가 : {buy}, 매도가 : {sell}, 수익율 : {rtn}")
        # 누적수익율 계산
        acc_rtn *= rtn

    return acc_rtn


    

In [93]:
aapl = pd.read_csv('../../csv/AAPL.csv')

In [95]:
aapl['Date'] = pd.to_datetime(aapl['Date'])
aapl.set_index('Date', inplace=True)
aapl.loc['2000-11']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-11-01,1.388393,1.491071,1.388393,1.464286,1.282757,143841600.0
2000-11-02,1.508929,1.602679,1.504464,1.593750,1.396171,147673400.0
2000-11-03,1.642857,1.642857,1.566964,1.589286,1.392261,128955400.0
2000-11-06,1.602679,1.616071,1.491071,1.531250,1.341420,98369600.0
2000-11-07,1.535714,1.558036,1.486607,1.522321,1.333598,75490800.0
2000-11-08,1.526786,1.531250,1.415179,1.433036,1.255381,105522200.0
2000-11-09,1.419643,1.464286,1.361607,1.441964,1.263203,119208600.0
2000-11-10,1.382807,1.419643,1.361607,1.361607,1.192808,105562800.0
2000-11-13,1.339286,1.428571,1.303571,1.383929,1.212362,107954000.0


In [88]:
six_month(aapl, 2000, 2019, 11)

매수가 : 1.388393, 매도가 : 1.820714, 수익율 : 1.3113822959349406
매수가 : 1.260714, 매도가 : 1.733571, 수익율 : 1.375070793217177
매수가 : 1.138571, 매도가 : 1.015714, 수익율 : 0.8920954424449595
매수가 : 1.630714, 매도가 : 1.841429, 수익율 : 1.1292164045933253
매수가 : 3.75, 매도가 : 5.151429, 수익율 : 1.3737144000000001
매수가 : 8.177143, 매도가 : 10.055715, 수익율 : 1.2297345173002356
매수가 : 11.585714, 매도가 : 14.257143, 수익율 : 1.2305795741203347
매수가 : 26.942858, 매도가 : 24.85, 수익율 : 0.9223223460554927
매수가 : 15.132857, 매도가 : 17.975714, 수익율 : 1.1878598998193137
매수가 : 27.114286, 매도가 : 37.298573, 수익율 : 1.3756059444087887
매수가 : 43.174286, 매도가 : 50.01857, 수익율 : 1.1585268601778382
매수가 : 56.772858, 매도가 : 83.425713, 수익율 : 1.4694647396472449
매수가 : 85.459999, 매도가 : 63.254284, 수익율 : 0.7401624706314355
매수가 : 74.860001, 매도가 : 84.298569, 수익율 : 1.1260829264482644
매수가 : 108.220001, 매도가 : 125.150002, 수익율 : 1.1564405917904215
매수가 : 120.800003, 매도가 : 93.739998, 수익율 : 0.7759933416557945
매수가 : 113.459999, 매도가 : 143.649994, 수익율 : 1.2660849221407098
매수가 : 169.869

8.177103601339539

In [76]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,2.4375,2.5,1.927083,1.958333,1.958333,72156000


In [77]:
df2 = pd.read_csv("../../csv/AAPL.csv")
df2.head(1)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0


In [89]:
df2['Date'] = pd.to_datetime(df2['Date'])

In [90]:
df2.set_index('Date', inplace=True)

In [91]:
df2.loc['2000-11']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-11-01,1.388393,1.491071,1.388393,1.464286,1.282757,143841600.0
2000-11-02,1.508929,1.602679,1.504464,1.593750,1.396171,147673400.0
2000-11-03,1.642857,1.642857,1.566964,1.589286,1.392261,128955400.0
2000-11-06,1.602679,1.616071,1.491071,1.531250,1.341420,98369600.0
2000-11-07,1.535714,1.558036,1.486607,1.522321,1.333598,75490800.0
2000-11-08,1.526786,1.531250,1.415179,1.433036,1.255381,105522200.0
2000-11-09,1.419643,1.464286,1.361607,1.441964,1.263203,119208600.0
2000-11-10,1.382807,1.419643,1.361607,1.361607,1.192808,105562800.0
2000-11-13,1.339286,1.428571,1.303571,1.383929,1.212362,107954000.0


In [80]:
# 컬럼에 Date가 존재하는가?
'Date' in df.columns

False

In [81]:
import yfinance as yf

In [83]:
aapl = yf.download('AAPL', start = '2018-01-01')

[*********************100%***********************]  1 of 1 completed


In [84]:
aapl.index

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-16',
               ...
               '2023-11-17', '2023-11-20', '2023-11-21', '2023-11-22',
               '2023-11-24', '2023-11-27', '2023-11-28', '2023-11-29',
               '2023-11-30', '2023-12-01'],
              dtype='datetime64[ns]', name='Date', length=1490, freq=None)

In [85]:
pd.to_datetime(aapl.index)

DatetimeIndex(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
               '2018-01-08', '2018-01-09', '2018-01-10', '2018-01-11',
               '2018-01-12', '2018-01-16',
               ...
               '2023-11-17', '2023-11-20', '2023-11-21', '2023-11-22',
               '2023-11-24', '2023-11-27', '2023-11-28', '2023-11-29',
               '2023-11-30', '2023-12-01'],
              dtype='datetime64[ns]', name='Date', length=1490, freq=None)

In [99]:
# 투자 기간은 고정 투자 월을 1월 ~ 12월 수익율 비교
df = yf.download('GOOG', start='2009-01-01')
month_rtns = []

for i in range(1, 13, 1):
    rtn = six_month(df, 2009, 2019, i)
    month_rtns.append(rtn)

month_rtns

[*********************100%***********************]  1 of 1 completed


[1.0543809925788026,
 2.173266872452559,
 1.71736804647562,
 2.4587242797213382,
 4.568705112506093,
 3.9235951237178646,
 6.06033759151672,
 2.862781223580768,
 3.8872710511276565,
 2.78491330473704,
 1.351706291505096,
 1.2803204936281518]